In [1]:
# https://medium.com/@miloszivic99/finetuning-large-language-models-customize-llama-3-8b-for-your-needs-bfe0f43cd239
import re 
import os
import os.path as osp
import torch
import pandas
import pandas as pd
import json
from typing import Union
from typing import List
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

bin C:\Users\any\anaconda3\envs\oosij\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
c_path = './data/classification/ukairia777/finance_data.csv' # 분류 .csv
s_path = './data/summarization/aihub_news' # 요약 폴더내.json 들
template_path = './templates/multi.json' # 템플릿  .json
dataset_name = "Smoked-Salmon-s/empathetic_dialogues_ko" # 싱글/멀티 대화형 데이터셋  허깅페이스 :

model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

name = model_name.split('/')[1]
new_model = name + "_train_v5" 
new_model

'Llama-3-Open-Ko-8B-Instruct-preview_train_v5'

In [ ]:
## Model Load 

In [3]:
save_path = 'models/' +  new_model + '_eval'
save_path

'models/Llama-3-Open-Ko-8B-Instruct-preview_train_v5_eval'

In [4]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= 'auto',
)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
#  추론용
def inference_output(prompt):
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
    # 인퍼런스 커스텀
    #inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda") # return_token_type_ids=False
    # peft 일시, **tokenizer(prompt, return_tensors="pt", add_special_tokens=False) 아래 inputs['input_ids'] 교체 , 위는 삭제 
    output = model.generate(
        #**tokenizer(prompt, return_tensors="pt", add_special_tokens=False),
        input_ids,
        max_new_tokens = 512, # 200  256 512
        do_sample= True, 
        top_p= 0.9,
        #top_k = 1, 
        temperature= 1, # 0.37,
        #early_stopping= True,
        eos_token_id=terminators,  
    )
    
    output = output[0].to("cpu")
    outputs = tokenizer.decode(output)
    return  outputs

In [11]:
## 최종 챗
import warnings
# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore", message="The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.")
warnings.filterwarnings("ignore", message="Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.")
# 트랜스포머 경고 메시지를 무시하도록 설정
logging.set_verbosity_error()

sys_msg = "당신은 업무적으로 도움을 주고 공감 능력이 있는 AI 봇 픽시입니다. 다음 대화의 흐름을 보고 상대방의 요청에 최대한 자세하고 친절하게 답해주세요."

messages = [
    {"role": "system", "content":sys_msg}
]

eos_token = '<|eot_id|>'

split_answer = '<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

while True : 
    user_input = input('[your msg] = ')
    if user_input == '0':
        print('[Pixy msg] =  대화를 종료합니다.')
        break
    user_prompt = {"role": "user", "content": user_input}  
    messages.append(user_prompt)
    output = inference_output(messages)
    response= output.split(split_answer)[-1].strip().replace(eos_token, '')
    print('[Pixy msg] = ', response)
    bot_output = {"role": "assistant", "content": response}
    messages.append(bot_output)

[your msg] =  안녕하세요?


[Pixy msg] =  안녕하세요! 저는 AI 봇 픽시입니다. 당신의 일상에서 가장 기억에 남는 순간이 무엇인지 알려주실 수 있나요?


[your msg] =  지금 더운 것이 가장 기억에 남네요.  관련된 뉴스를 요약해주시겠어요?     "목요일인 20일 중부 지방과 전북권, 경북권의 낮 기온이 33도 안팎으로 오르면서 덥겠다.     전남권과 경남권, 제주도는 비가 내리거나 구름이 짙게 끼면서 낮 기온이 전날보다 2∼6도가량 떨어지겠다.      전국 대부분 지역에서 체감온도가 최고 31도 이상 오르며 무더운 날씨를 보인 11일 서울 영등포구 여의대로에 지열로 인한 아지랑이가 피어오르고 있다. 뉴스1    이날 오전 5시 현재 기온은 서울 23.7도, 인천 22.0도, 수원 19.8도, 춘천 19.1도, 강릉 25.7도, 청주 22.9도, 대전 20.3도, 전주 20.8도, 광주 21.8도, 제주 22.2도, 대구 21.3도, 부산 21.6도, 울산 19.8도, 창원 20.2도 등이다.     낮 최고기온은 25∼35도로 예보됐다.     이날 전국에 가끔 구름이 많은 가운데 전남권과 제주도에 비가 내리겠다.     특히 제주도는 돌풍과 천둥·번개를 동반한 시간당 30㎜ 이상의 매우 강한 장맛비가 오겠다.  오전에 전북 남부와 경남권도 가끔 비가 오는 곳이 있겠으나 밤에 대부분 그치겠다.     예상 강수량은 전남 남해안 5∼20㎜, 경남 남해안 5∼10㎜, 광주·전남, 부산·울산·경남 내륙 5㎜ 안팎, 전북 남부 5㎜ 미만이다.     제주도는 이튿날 아침까지 50∼100㎜(많은 곳 150㎜ 이상, 산지 200㎜ 이상)의 비가 예보됐다.     미세먼지 농도는 전국이 ‘좋음’∼‘보통’ 수준으로 예상된다.     다만 수도권은 전날 잔류 미세먼지가 대기 정체와 기류 수렴으로 축적되며 오전에 잠시 ‘나쁨’ 수준을 보이겠다."


[Pixy msg] =  목요일인 20일 중부 지방과 전북권, 경북권의 낮 기온이 33도 안팎으로 오르면서 덥겠다. 전남권과 경남권, 제주도는 비가 내리거나 구름이 짙게 끼면서 낮 기온이 전날보다 2∼6도가량 떨어지겠다. 전국 대부분 지역에서 체감온도가 최고 31도 이상 오르며 무더운 날씨를 보인 11일 서울 영등포구 여의대로에 지열로 인한 아지랑이가 피어오르고 있다.


[your msg] =  좋습니다. 다음으로 그 요약한 것을 감정 분류해주세요. 라벨은 "긍정", "부정", "중립" 3가지 중 한가지로 대답해주세요.


...

[Pixy msg] =  네, 저는 당신의 일상에서 불편한 점을 찾아보고 해결책을 제안하는 등의 일을 도와드릴 수 있습니다.


[your msg] =  좋습니다. 하지만 그 기회는 다음으로 미뤄야 할 것 같아요. 대화를 이만 종료합시다.


[Pixy msg] =  알겠습니다. 다음에 만나서 더 즐거운 대화를 나누도록 하겠습니다. 그럼 이만.


[your msg] =  0


[Pixy msg] =  대화를 종료합니다.
